In [2]:
import transformers
from transformers import pipeline

clf = pipeline('sentiment-analysis')
result = clf('what a beautiful day')[0]
print("감성 분석 결과 : %s, 감성스코어 : %0.4f" % (result['label'], result['score']))

2024-06-03 19:08:24.166510: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 19:08:24.202492: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 19:08:24.484390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 19:08:25.596779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99

감성 분석 결과 : POSITIVE, 감성스코어 : 0.9999


In [3]:
from transformers import pipeline

text_generater = pipeline('text-generation')
result = text_generater('Alice was beginning to get very tired of sitting by her sister on the bank,')
print(result[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Alice was beginning to get very tired of sitting by her sister on the bank, so she went to a nearby bar. She took a long sip from her drink and quickly pulled out a small book, "Nymphomancy," with a subtitle.


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained(
"bert-base-cased-finetuned-mrpc"
)

input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"

tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")

logits = model(**tokens).logits

results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 29%
yes: 71%


In [5]:
target_sequence = "The boy quickly ran across the finish line, seizing yet another victory"

tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
logits = model(**tokens).logits
results = torch.softmax(logits, dim=1).tolist()[0]
for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 95%
yes: 5%


In [6]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np
nltk.download('movie_reviews')

fileids = movie_reviews.fileids()

reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])
X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/hyna/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count:  1600
Test set count:  400


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained(
"distilbert-base-uncased-finetuned-sst-2-english"
)

model = AutoModelForSequenceClassification.from_pretrained(
"distilbert-base-uncased-finetuned-sst-2-english"
)


model = model.to(device)

batch_size = 10 
y_pred = [] 
num_batch = len(y_test)//batch_size

for i in range(num_batch):
    inputs = tokenizer(
        X_test[i*batch_size:(i+1)*batch_size],
        truncation=True,
        padding=True,
        return_tensors="pt"
    )  
    
    inputs = inputs.to(device)
    
    logits = model(**inputs).logits
    
    pred = F.softmax(logits, dim=-1)
    
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    y_pred.extend(results.tolist())

torch.cuda.empty_cache()

score = sum(y_test = np.array(y_pred))/len(y_test)
print("NLTK 영화리뷰 감성분석 정확도:", score)


: 